In [322]:
import matplotlib as plt
import findspark
findspark.init()

import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType
from pyspark.sql.functions import to_date, mean, col, lit, concat, unix_timestamp, month, year, regexp_replace, round
spark = SparkSession.builder.getOrCreate()

In [323]:
df =  spark.read.csv(
      path="SEMANAL_MUNICIPIOS-2019.csv",
      sep=",",
      header=True,
      quote='"',
      inferSchema=True
)
df = df.withColumn('PREÇO MÉDIO REVENDA', regexp_replace('PREÇO MÉDIO REVENDA', ',', '.').cast('float'))
df.printSchema()

root
 |-- DATA INICIAL: string (nullable = true)
 |-- DATA FINAL: string (nullable = true)
 |-- REGIÃO: string (nullable = true)
 |-- ESTADO: string (nullable = true)
 |-- MUNICÍPIO: string (nullable = true)
 |-- PRODUTO: string (nullable = true)
 |-- NÚMERO DE POSTOS PESQUISADOS: integer (nullable = true)
 |-- UNIDADE DE MEDIDA: string (nullable = true)
 |-- PREÇO MÉDIO REVENDA: float (nullable = true)
 |-- DESVIO PADRÃO REVENDA: string (nullable = true)
 |-- PREÇO MÍNIMO REVENDA: string (nullable = true)
 |-- PREÇO MÁXIMO REVENDA: string (nullable = true)
 |-- MARGEM MÉDIA REVENDA: string (nullable = true)
 |-- COEF DE VARIAÇÃO REVENDA: string (nullable = true)
 |-- PREÇO MÉDIO DISTRIBUIÇÃO: string (nullable = true)
 |-- DESVIO PADRÃO DISTRIBUIÇÃO: string (nullable = true)
 |-- PREÇO MÍNIMO DISTRIBUIÇÃO: string (nullable = true)
 |-- PREÇO MÁXIMO DISTRIBUIÇÃO: string (nullable = true)
 |-- COEF DE VARIAÇÃO DISTRIBUIÇÃO: string (nullable = true)



In [324]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df = df.orderBy('DATA INICIAL')

In [325]:
df = (df.withColumn('DATA INICIAL', to_date(unix_timestamp('DATA INICIAL', 'dd/MM/yyyy').cast('timestamp'))))
df = (df.withColumn('Mês', month('DATA INICIAL')))
df = (df.withColumn('Ano', year('DATA INICIAL')))
df.describe()

DataFrame[summary: string, DATA FINAL: string, REGIÃO: string, ESTADO: string, MUNICÍPIO: string, PRODUTO: string, NÚMERO DE POSTOS PESQUISADOS: string, UNIDADE DE MEDIDA: string, PREÇO MÉDIO REVENDA: string, DESVIO PADRÃO REVENDA: string, PREÇO MÍNIMO REVENDA: string, PREÇO MÁXIMO REVENDA: string, MARGEM MÉDIA REVENDA: string, COEF DE VARIAÇÃO REVENDA: string, PREÇO MÉDIO DISTRIBUIÇÃO: string, DESVIO PADRÃO DISTRIBUIÇÃO: string, PREÇO MÍNIMO DISTRIBUIÇÃO: string, PREÇO MÁXIMO DISTRIBUIÇÃO: string, COEF DE VARIAÇÃO DISTRIBUIÇÃO: string, Mês: string, Ano: string]

In [330]:
df = df.orderBy("Ano", "Mês", "Município", "Produto")
exe1 = (df.groupBy("Ano","Mês","Município","Produto","Unidade de Medida").agg(round(mean("PREÇO MÉDIO REVENDA").alias("Preço Médio"),3).alias("Preço médio"))).show()

+----+---+--------------------+----------------+-----------------+-----------+
| Ano|Mês|           Município|         Produto|Unidade de Medida|Preço médio|
+----+---+--------------------+----------------+-----------------+-----------+
|2018| 12|          ABAETETUBA|ETANOL HIDRATADO|             R$/l|       4.65|
|2018| 12|          ABAETETUBA|  GASOLINA COMUM|             R$/l|      4.955|
|2018| 12|          ABAETETUBA|             GLP|          R$/13Kg|     70.667|
|2018| 12|          ABAETETUBA|     ÓLEO DIESEL|             R$/l|      4.088|
|2018| 12|          ABAETETUBA| ÓLEO DIESEL S10|             R$/l|       4.12|
|2018| 12|          ACAILANDIA|ETANOL HIDRATADO|             R$/l|      3.524|
|2018| 12|          ACAILANDIA|  GASOLINA COMUM|             R$/l|      4.189|
|2018| 12|          ACAILANDIA|             GLP|          R$/13Kg|       74.0|
|2018| 12|          ACAILANDIA|     ÓLEO DIESEL|             R$/l|      3.538|
|2018| 12|          ACAILANDIA| ÓLEO DIESEL S10|    

In [333]:
df = df.orderBy("Estado","Região")
exe2 =(df.groupBy("Estado","Região",).agg(round(mean("PREÇO MÉDIO REVENDA").alias("Preço Médio"),3).alias("Preço médio"))).show()

+-------------------+------------+-----------+
|             Estado|      Região|Preço médio|
+-------------------+------------+-----------+
|               ACRE|       NORTE|     19.451|
|            ALAGOAS|    NORDESTE|     15.369|
|              AMAPA|       NORTE|     21.808|
|           AMAZONAS|       NORTE|     20.866|
|              BAHIA|    NORDESTE|     15.238|
|              CEARA|    NORDESTE|     17.457|
|   DISTRITO FEDERAL|CENTRO OESTE|     17.239|
|     ESPIRITO SANTO|     SUDESTE|     14.959|
|              GOIAS|CENTRO OESTE|     17.045|
|           MARANHAO|    NORDESTE|     18.291|
|        MATO GROSSO|CENTRO OESTE|     22.364|
| MATO GROSSO DO SUL|CENTRO OESTE|     17.522|
|       MINAS GERAIS|     SUDESTE|     16.962|
|               PARA|       NORTE|     20.842|
|            PARAIBA|    NORDESTE|     15.817|
|             PARANA|         SUL|     16.837|
|         PERNAMBUCO|    NORDESTE|     17.882|
|              PIAUI|    NORDESTE|      18.08|
|     RIO DE 